## GEOGRAPHICAL FEATURE VISUALIZATION

In [1]:
import folium
import geopandas as gpd
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from folium import Choropleth , Circle , Marker , Icon ,Map

from folium.plugins import HeatMap, MarkerCluster

In [2]:
claimsdf = pd.read_csv('/home/julian/Cursos/Ironhack/Proyectos/ProyectoFinal/Claims-Frequency-Predictions/Notebooks/claimsdf_1.csv')

In [66]:
claimsdf = pd.read_csv('/home/julian/Cursos/Ironhack/Proyectos/ProyectoFinal/Claims-Frequency-Predictions/freMTPL2freq.csv')

In [67]:
claimsdf['empirical_frequencies'] = (claimsdf.ClaimNb / claimsdf.Exposure)

In [68]:
def claims_(x):
    """
    arg: A claim count
    returns:if x is bigger than four, returns a four, otherwise, returns the same value
    """
    if x > 4 :
        return 4
    else: 
        return x

In [69]:
claimsdf['ClaimNb'] = claimsdf.ClaimNb.apply(lambda x: claims_(x))

In [70]:
regions = ['Île_de_France', 'Champagne_Ardenne', 'Picardie', 'Haute_Normandie', 'Centre', 'Basse_Normandie', 'Bourgogne', 'Nord_Pas-de-Calais', 'Lorraine', 'Alsace', 'Franche_Comté', 'Pays_de_la_Loire', 'Bretagne', 'Poitou_Charentes', 'Aquitaine', 'Midi_Pyrénées', 'Limousin', 'Rhône_Alpes', 'Auvergne', 'Languedoc_Roussillon', 'Provence-Alpes_Côte d´Azur', 'Corse']
regions_code = ['R11', 'R21', 'R22', 'R23', 'R24', 'R25', 'R26', 'R31', 'R41', 'R42', 'R43', 'R52', 'R53', 'R54', 'R72', 'R73', 'R74', 'R82', 'R83', 'R91', 'R93', 'R94']
for i,j in zip(regions_code, regions):
    claimsdf.Region = claimsdf.Region.str.replace(i, j) 

In [71]:
new_regions = {'Île_de_France': 'Île-de-France',
               'Centre': 'Centre-Val de Loire',
               'Bourgogne': 'Bourgogne-Franche-Comté',
               'Franche_Comté': 'Bourgogne-Franche-Comté',
               'Haute_Normandie': 'Normandie',
               'Basse_Normandie': 'Normandie',
               'Nord_Pas-de-Calais': 'Hauts-de-France',
               'Picardie': 'Hauts-de-France',
               'Champagne_Ardenne': 'Grand Est',
               'Lorraine': 'Grand Est',
               'Alsace': 'Grand Est',
               'Pays_de_la_Loire': 'Pays de la Loire',
               'Bretagne': 'Bretagne', 
               'Aquitaine': 'Nouvelle-Aquitaine',
               'Limousin': 'Nouvelle-Aquitaine',
               'Poitou_Charentes': 'Nouvelle-Aquitaine',               
               'Languedoc_Roussillon': 'Occitanie',
               'Midi_Pyrénées': 'Occitanie',                             
               'Rhône_Alpes': 'Auvergne-Rhône-Alpes',
               'Auvergne': 'Auvergne-Rhône-Alpes',
               'Provence-Alpes_Côte d´Azur':"Provence-Alpes-Côte d'Azur",              
               'Corse': 'Corse'}

In [72]:
claimsdf['Region'] = claimsdf.Region.map(new_regions)

In [73]:
claimsdf['log_density'] = np.log(claimsdf.Density) 

In [74]:
claimsdf.drop(columns=['IDpol', 'Density'], inplace=True)

In [75]:
agrup = claimsdf.groupby('Region').agg({'ClaimNb': 'mean', 'Exposure': 'mean', 'BonusMalus': 'mean', 'empirical_frequencies': 'mean', 'VehPower': 'mean', 'VehAge': 'mean', 'DrivAge': 'mean', 'log_density': 'mean'})

In [76]:
agrup.reset_index(inplace=True)

In [77]:
agrup

,Region,ClaimNb,Exposure,BonusMalus,empirical_frequencies,VehPower,VehAge,DrivAge,log_density
0,Auvergne-Rhône-Alpes,0.057986,0.529441,60.246027,0.268019,6.593410,7.048246,44.848999,6.498102
1,Bourgogne-Franche-Comté,0.048062,0.472954,60.914029,0.331357,6.451345,6.042647,44.590878,4.963646
2,Bretagne,0.064147,0.658883,57.619273,0.213853,6.307796,7.729144,46.504226,5.625863
3,Centre-Val de Loire,0.057266,0.639553,58.835617,0.209384,6.332819,8.952460,46.137315,4.810377
4,Corse,0.054694,0.391040,61.106732,0.379998,6.713020,3.869353,45.903897,5.066657
5,Grand Est,0.049627,0.577993,58.056599,0.262524,6.296223,6.241107,47.398551,6.118278
6,Hauts-de-France,0.045778,0.427234,63.369313,0.266500,6.277814,6.107571,41.120922,6.475907
7,Normandie,0.047568,0.499852,59.497179,0.219722,6.437262,6.922244,44.739239,5.862101
8,Nouvelle-Aquitaine,0.046868,0.507473,59.413090,0.233871,6.441611,7.144170,45.705253,5.525214
9,Occitanie,0.041495,0.410913,59.628508,0.289101,6.453368,5.777736,45.768783,5.704623


In [79]:
prueba = agrup.to_dict("records")

In [86]:
prueba[0]

{'Region': 'Auvergne-Rhône-Alpes',
 'ClaimNb': 0.05798598385144215,
 'Exposure': 0.5294405925688708,
 'BonusMalus': 60.24602672175391,
 'empirical_frequencies': 0.26801919068214647,
 'VehPower': 6.593409522540233,
 'VehAge': 7.048245760170593,
 'DrivAge': 44.84899876720088,
 'log_density': 6.498102225578569}

In [89]:
nueva = []
for value in new_regions.values():
    for elemento in prueba:
        for k,v in elemento.items():
            if k == "Region":
                if value == v:
                    nueva.append(elemento)

In [90]:
nueva

[{'Region': 'Île-de-France',
  'ClaimNb': 0.0569987534209282,
  'Exposure': 0.43284437756247307,
  'BonusMalus': 62.52177214827127,
  'empirical_frequencies': 0.4131051613502608,
  'VehPower': 6.722757948732645,
  'VehAge': 4.801392729721598,
  'DrivAge': 44.624292530555515,
  'log_density': 8.106634051351211},
 {'Region': 'Centre-Val de Loire',
  'ClaimNb': 0.05726614404642561,
  'Exposure': 0.6395533239872213,
  'BonusMalus': 58.835617461908704,
  'empirical_frequencies': 0.20938397786784452,
  'VehPower': 6.332818600133249,
  'VehAge': 8.952459822790642,
  'DrivAge': 46.137315458807855,
  'log_density': 4.810376968147281},
 {'Region': 'Bourgogne-Franche-Comté',
  'ClaimNb': 0.04806227788119817,
  'Exposure': 0.47295410014399974,
  'BonusMalus': 60.91402944660687,
  'empirical_frequencies': 0.3313568667029486,
  'VehPower': 6.451345405313928,
  'VehAge': 6.042646809950923,
  'DrivAge': 44.59087832120494,
  'log_density': 4.963645524605266},
 {'Region': 'Bourgogne-Franche-Comté',
  'C

In [91]:
agrup2 = pd.DataFrame(prueba)

In [93]:
agrup2

,Region,ClaimNb,Exposure,BonusMalus,empirical_frequencies,VehPower,VehAge,DrivAge,log_density
0,Auvergne-Rhône-Alpes,0.057986,0.529441,60.246027,0.268019,6.593410,7.048246,44.848999,6.498102
1,Bourgogne-Franche-Comté,0.048062,0.472954,60.914029,0.331357,6.451345,6.042647,44.590878,4.963646
2,Bretagne,0.064147,0.658883,57.619273,0.213853,6.307796,7.729144,46.504226,5.625863
3,Centre-Val de Loire,0.057266,0.639553,58.835617,0.209384,6.332819,8.952460,46.137315,4.810377
4,Corse,0.054694,0.391040,61.106732,0.379998,6.713020,3.869353,45.903897,5.066657
5,Grand Est,0.049627,0.577993,58.056599,0.262524,6.296223,6.241107,47.398551,6.118278
6,Hauts-de-France,0.045778,0.427234,63.369313,0.266500,6.277814,6.107571,41.120922,6.475907
7,Normandie,0.047568,0.499852,59.497179,0.219722,6.437262,6.922244,44.739239,5.862101
8,Nouvelle-Aquitaine,0.046868,0.507473,59.413090,0.233871,6.441611,7.144170,45.705253,5.525214
9,Occitanie,0.041495,0.410913,59.628508,0.289101,6.453368,5.777736,45.768783,5.704623


In [ ]:
"""dic_region = {'Auvergne-Rhône-Alpes': 1, 'Bourgogne-Franche-Comté': 2, 'Centre-Val de Loire': 3, 'Grand Est': 4,
               'Hauts-de-France': 5, 'Normandie': 6, 'Nouvelle-Aquitaine': 7, 'Occitanie': 8, 'Bretagne': 9,
               'Corse': 10, 'Île-de-France': 11, 'Pays de la Loire': 12, "Provence-Alpes-Côte d'Azur": 13}
claimsdf['Region_'] = claimsdf.Region.map(dic_region)
"""

### GEOS

In [6]:
geo_json = r'regions.geojson'

with open (geo_json) as geo_file:
    geo_france =  json.load(geo_file)

In [7]:
"""
#gdf = gpd.GeoDataFrame.from_features(geo_france["features"])
#O..

gdf.to_file('regions.geojson',driver='GeoJSON')

gdf.crs = "EPSG:4326"
gdf = gdf.to_crs("EPSG:3857")
print(gdf.crs)
"""

'\n#gdf = gpd.GeoDataFrame.from_features(geo_france["features"])\n#O..\n\ngdf.to_file(\'regions.geojson\',driver=\'GeoJSON\')\n\ngdf.crs = "EPSG:4326"\ngdf = gdf.to_crs("EPSG:3857")\nprint(gdf.crs)\n'

In [57]:
geo_france["features"][1]["properties"]["nom"]

'Centre-Val de Loire'

In [58]:
for i in range(len(geo_france.get("features"))):
    print(geo_france.get("features")[i].get("properties"))

{'code': '11', 'nom': 'Île-de-France'}
{'code': '24', 'nom': 'Centre-Val de Loire'}
{'code': '27', 'nom': 'Bourgogne-Franche-Comté'}
{'code': '28', 'nom': 'Normandie'}
{'code': '32', 'nom': 'Hauts-de-France'}
{'code': '44', 'nom': 'Grand Est'}
{'code': '52', 'nom': 'Pays de la Loire'}
{'code': '53', 'nom': 'Bretagne'}
{'code': '75', 'nom': 'Nouvelle-Aquitaine'}
{'code': '76', 'nom': 'Occitanie'}
{'code': '84', 'nom': 'Auvergne-Rhône-Alpes'}
{'code': '93', 'nom': "Provence-Alpes-Côte d'Azur"}
{'code': '94', 'nom': 'Corse'}


In [59]:
france_regions_map = folium.Map(location=[46.72213865950384,2.6911389975442024], tiles='CartoDB dark_matter', zoom_start=6)
france_regions_map

In [64]:
france_regions_map.choropleth(
    geo_data = geo_json, 
    data = agrup, 
    columns = ["Region", "log_density"], 
    key_on = "feature.properties.nom", 
    fill_color = "YlGnBu", 
    fill_opacity = 1, 
    line_opacity = 1, 
    legend_name = "Regions", 
    smooth_factor = 0)

/home/julian/miniconda3/envs/ironcon/lib/python3.8/site-packages/folium/folium.py:409: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


In [65]:
france_regions_map

In [13]:
france_regions_map.save("choro.jpg")

#### Now, we proceed to export a csv file with the wrangled data frame for mapping

In [14]:
claimsdf.to_csv('claimsdf_3.csv', index = False)